In [3]:
import pandas as pd
import polars as pl

import os
from glob import glob

import lightgbm as lgb
from sklearn.model_selection import GroupKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import matplotlib.pyplot as plt

In [2]:
os.chdir('c:/Users/laura/OneDrive/Documentos/Personal Documents/Universidad/DSE CCNY/Courses Semester 2/Applied ML/Final Project/machine-learning-dse-i210-final-project-credit-risk/notebooks')

In [3]:
from utils.merge_tools import merge_n_case_ids_batch_processing

In [4]:
data_dir = 'c:/Users/laura/OneDrive/Documentos/Personal Documents/Universidad/DSE CCNY/Courses Semester 2/Applied ML/Final Project/new_aggs/new_aggs/'
data_base = 'C:/Users/laura/OneDrive/Documentos/Personal Documents/Universidad/DSE CCNY/Courses Semester 2/Applied ML/Project_final/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv'

# Personal Data Model

Reading from the pre-processed files 100 samples rows from the personal data. 

In [5]:
df = merge_n_case_ids_batch_processing(
    n_ids=30,
    data_dir=data_dir,
    path_to_base=data_base,
    use_0=True, 
    as_pandas=False, 
    random_state=42,
    batch_size=15
)

### Train and Validation Split

In [2]:
# Train and Validation split

base_train = df columns "case_id", "WEEK_NUM", "target"
X = df all columns except "case_id", "WEEK_NUM", "target"
y = df column targrt


X_train, X_valid = train_test_split(X, train_size=0.7, random_state=1)

print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")


### Training a LightGBM Model on Personal Data

In [ ]:
## Training LightGBM
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

### Scores

Evaluation with AUC and then comparison with the stability metric is shown below.

In [ ]:
for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 

Stability Metric

In [ ]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 